# Job Scheduler

## Overview:

- **Teaching:** 30 min
- **Exercises:** 30 min

**Questions**
- What is a scheduler and why are they used?
- How do I launch a program to run on a compute node?
- How do I check limits on the cluster and my accounts?

**Objectives**
- Run a simple job on the cluster's compute nodes.
- Inspect the status of your job.
- Inspect the output and error files of your job.